# 1.欠完备自编码器
(1)实现欠完备自编码器
(2)修改不同大小的隐藏层，观察自编码器的输出表现

# 2.降噪自编码器
实现降噪自编码器

In [8]:
import tensorflow as tf 
import numpy as np 
from tensorflow.examples.tutorials.mnist import input_data
mnist_width = 28 
n_visible = mnist_width * mnist_width 
n_hidden = 500 
corruption_level = 0.3 
# 输入的一张图片用28x28=784的向量表示. 
X = tf.placeholder("float", [None, n_visible], name='X') 
# 用于将部分输入数据置为0 
mask = tf.placeholder("float", [None, n_visible], name='mask') 
# create nodes for hidden variables 
W_init_max = 4 * np.sqrt(6. / (n_visible + n_hidden)) 
W_init = tf.random_uniform(shape=[n_visible, n_hidden], 
                           minval=-W_init_max, 
                           maxval=W_init_max) 
# 编码器 
W = tf.Variable(W_init, name='W')#shape:784x500 
b = tf.Variable(tf.zeros([n_hidden]), name='b')#隐含层的偏置 
#解码器 
W_prime = tf.transpose(W) 
b_prime = tf.Variable(tf.zeros([n_visible]), name='b_prime') 
def model(X, mask, W, b, W_prime, b_prime): 
    tilde_X = mask * X # corrupted X 
    Y = tf.nn.sigmoid(tf.matmul(tilde_X, W) + b) # hidden state 
    Z = tf.nn.sigmoid(tf.matmul(Y, W_prime) + b_prime) # reconstructed input 
    return Z 
# build model graph 
Z = model(X, mask, W, b, W_prime, b_prime) 
# create cost function 
cost = tf.reduce_sum(tf.pow(X - Z, 2)) # minimize squared error 
train_op = tf.train.GradientDescentOptimizer(0.02).minimize(cost) # construct an optimizer 
# load MNIST data 
mnist = input_data.read_data_sets("MNIST_data", one_hot=True) 
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels 
# Launch the graph in a session 
with tf.Session() as sess: 
    # you need to initialize all variables 
    tf.initialize_all_variables().run() 
    for i in range(100): 
        for start, end in zip(range(0, len(trX), 128), range(128, len(trX)+1, 128)): 
            input_ = trX[start:end] 
            mask_np = np.random.binomial(1, 1 - corruption_level, input_.shape) 
            sess.run(train_op, feed_dict={X: input_, mask: mask_np})
        mask_np = np.random.binomial(1, 1 - corruption_level, teX.shape) 
        print(i, sess.run(cost, feed_dict={X: teX, mask: mask_np}))


Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.
0 111372.87
1 93316.76
2 88167.766
3 81623.95
4 79598.55
5 74131.31
6 73553.86
7 73230.06
8 70209.11
9 69576.88
10 67659.31
11 67723.125
12 65237.91
13 65427.918
14 65031.75
15 64403.055
16 63778.234
17 63997.44
18 63401.535
19 62778.453
20 62671.27
21 62785.492
22 62226.426
23 61226.83
24 62245.402
25 61097.504
26 61392.45
27 61229.617
28 60494.234
29 59681.383
30 60667.562
31 61433.957
32 60017.777
33 60476.96
34 59946.547
35 59718.58
36 59597.188
37 59615.527
38 60335.84
39 58954.855
40 59370.734
41 59155.145
42 59554.46
43 58441.566
44 59221.79
45 58429.645
46 58665.375
47 59069.312
48 58185.844
49 57904.79
50 58258.188
51 58115.79
52 59674.266
53 58402.195
54 58305.133
55 58259.223
56 58049.055
57 58160.523
58 57455.0

# 3.卷积自编码器
(1)实现卷积自编码器
(2)使用卷积自编码器对图片进行降噪

In [9]:
learning_rate = 0.001
inputs_ = tf.placeholder(tf.float32, (None, 28, 28, 1), name='inputs')
targets_ = tf.placeholder(tf.float32, (None, 28, 28, 1), name='targets')
### Encoder
conv1 = tf.layers.conv2d(inputs=inputs_, filters=32, kernel_size=(3,3), padding='same', activation=tf.nn.relu)
# Now 28x28x32
maxpool1 = tf.layers.max_pooling2d(conv1, pool_size=(2,2), strides=(2,2), padding='same')
# Now 14x14x32
conv2 = tf.layers.conv2d(inputs=maxpool1, filters=32, kernel_size=(3,3), padding='same', activation=tf.nn.relu)
# Now 14x14x32
maxpool2 = tf.layers.max_pooling2d(conv2, pool_size=(2,2), strides=(2,2), padding='same')
# Now 7x7x32
conv3 = tf.layers.conv2d(inputs=maxpool2, filters=16, kernel_size=(3,3), padding='same', activation=tf.nn.relu)
# Now 7x7x16
encoded = tf.layers.max_pooling2d(conv3, pool_size=(2,2), strides=(2,2), padding='same')
# Now 4x4x16
### Decoder
upsample1 = tf.image.resize_images(encoded, size=(7,7), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
# Now 7x7x16
conv4 = tf.layers.conv2d(inputs=upsample1, filters=16, kernel_size=(3,3), padding='same', activation=tf.nn.relu)
# Now 7x7x16
upsample2 = tf.image.resize_images(conv4, size=(14,14), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
# Now 14x14x16
conv5 = tf.layers.conv2d(inputs=upsample2, filters=32, kernel_size=(3,3), padding='same', activation=tf.nn.relu)
# Now 14x14x32
upsample3 = tf.image.resize_images(conv5, size=(28,28), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
# Now 28x28x32
conv6 = tf.layers.conv2d(inputs=upsample3, filters=32, kernel_size=(3,3), padding='same', activation=tf.nn.relu)
# Now 28x28x32
logits = tf.layers.conv2d(inputs=conv6, filters=1, kernel_size=(3,3), padding='same', activation=None)
#Now 28x28x1
# Pass logits through sigmoid to get reconstructed image
decoded = tf.nn.sigmoid(logits)
# Pass logits through sigmoid and calculate the cross-entropy loss
loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=targets_, logits=logits)
# Get cost and define the optimizer
cost = tf.reduce_mean(loss)
opt = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [ ]:
epochs=100
batch_size = 200
with tf.Session() as sess:  
    # Set's how much noise we're adding to the MNIST images
    noise_factor = 0.5
    sess.run(tf.global_variables_initializer())
    for e in range(epochs):
        for ii in range(mnist.train.num_examples//batch_size):
            batch = mnist.train.next_batch(batch_size)
            # Get images from the batch
            imgs = batch[0].reshape((-1, 28, 28, 1))
        
            # Add random noise to the input images
            noisy_imgs = imgs + noise_factor * np.random.randn(*imgs.shape)
            # Clip the images to be between 0 and 1
            noisy_imgs = np.clip(noisy_imgs, 0., 1.)
        
            # Noisy images as inputs, original images as targets
            batch_cost, _ = sess.run([cost, opt], feed_dict={inputs_: noisy_imgs,
                                                         targets_: imgs})
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Training loss: {:.4f}".format(batch_cost))

Epoch: 1/100... Training loss: 0.6906
Epoch: 1/100... Training loss: 0.6749
Epoch: 1/100... Training loss: 0.6482
Epoch: 1/100... Training loss: 0.6126
Epoch: 1/100... Training loss: 0.5639
Epoch: 1/100... Training loss: 0.5229
Epoch: 1/100... Training loss: 0.5153
Epoch: 1/100... Training loss: 0.5241
Epoch: 1/100... Training loss: 0.5162
Epoch: 1/100... Training loss: 0.5088
Epoch: 1/100... Training loss: 0.4903
Epoch: 1/100... Training loss: 0.4802
Epoch: 1/100... Training loss: 0.4827
Epoch: 1/100... Training loss: 0.4771
Epoch: 1/100... Training loss: 0.4775
Epoch: 1/100... Training loss: 0.4617
Epoch: 1/100... Training loss: 0.4634
Epoch: 1/100... Training loss: 0.4423
Epoch: 1/100... Training loss: 0.4447
Epoch: 1/100... Training loss: 0.4319
Epoch: 1/100... Training loss: 0.4103
Epoch: 1/100... Training loss: 0.3959
Epoch: 1/100... Training loss: 0.3830
Epoch: 1/100... Training loss: 0.3823
Epoch: 1/100... Training loss: 0.3627
Epoch: 1/100... Training loss: 0.3534
Epoch: 1/100

Epoch: 1/100... Training loss: 0.1807
Epoch: 1/100... Training loss: 0.1774
Epoch: 1/100... Training loss: 0.1802
Epoch: 1/100... Training loss: 0.1772
Epoch: 1/100... Training loss: 0.1762
Epoch: 1/100... Training loss: 0.1824
Epoch: 1/100... Training loss: 0.1783
Epoch: 1/100... Training loss: 0.1778
Epoch: 1/100... Training loss: 0.1780
Epoch: 1/100... Training loss: 0.1741
Epoch: 1/100... Training loss: 0.1731
Epoch: 1/100... Training loss: 0.1762
Epoch: 1/100... Training loss: 0.1758
Epoch: 1/100... Training loss: 0.1784
Epoch: 1/100... Training loss: 0.1773
Epoch: 1/100... Training loss: 0.1792
Epoch: 1/100... Training loss: 0.1825
Epoch: 1/100... Training loss: 0.1772
Epoch: 1/100... Training loss: 0.1797
Epoch: 1/100... Training loss: 0.1771
Epoch: 1/100... Training loss: 0.1731
Epoch: 1/100... Training loss: 0.1679
Epoch: 1/100... Training loss: 0.1774
Epoch: 1/100... Training loss: 0.1722
Epoch: 1/100... Training loss: 0.1743
Epoch: 1/100... Training loss: 0.1773
Epoch: 1/100

Epoch: 2/100... Training loss: 0.1598
Epoch: 2/100... Training loss: 0.1601
Epoch: 2/100... Training loss: 0.1548
Epoch: 2/100... Training loss: 0.1560
Epoch: 2/100... Training loss: 0.1560
Epoch: 2/100... Training loss: 0.1590
Epoch: 2/100... Training loss: 0.1559
Epoch: 2/100... Training loss: 0.1556
Epoch: 2/100... Training loss: 0.1596
Epoch: 2/100... Training loss: 0.1581
Epoch: 2/100... Training loss: 0.1542
Epoch: 2/100... Training loss: 0.1556
Epoch: 2/100... Training loss: 0.1525
Epoch: 2/100... Training loss: 0.1517
Epoch: 2/100... Training loss: 0.1538
Epoch: 2/100... Training loss: 0.1552
Epoch: 2/100... Training loss: 0.1576
Epoch: 2/100... Training loss: 0.1480
Epoch: 2/100... Training loss: 0.1604
Epoch: 2/100... Training loss: 0.1527
Epoch: 2/100... Training loss: 0.1488
Epoch: 2/100... Training loss: 0.1552
Epoch: 2/100... Training loss: 0.1533
Epoch: 2/100... Training loss: 0.1599
Epoch: 2/100... Training loss: 0.1582
Epoch: 2/100... Training loss: 0.1551
Epoch: 2/100

Epoch: 3/100... Training loss: 0.1394
Epoch: 3/100... Training loss: 0.1341
Epoch: 3/100... Training loss: 0.1383
Epoch: 3/100... Training loss: 0.1441
Epoch: 3/100... Training loss: 0.1388
Epoch: 3/100... Training loss: 0.1420
Epoch: 3/100... Training loss: 0.1424
Epoch: 3/100... Training loss: 0.1403
Epoch: 3/100... Training loss: 0.1449
Epoch: 3/100... Training loss: 0.1397
Epoch: 3/100... Training loss: 0.1437
Epoch: 3/100... Training loss: 0.1434
Epoch: 3/100... Training loss: 0.1454
Epoch: 3/100... Training loss: 0.1444
Epoch: 3/100... Training loss: 0.1394
Epoch: 3/100... Training loss: 0.1467
Epoch: 3/100... Training loss: 0.1366
Epoch: 3/100... Training loss: 0.1382
Epoch: 3/100... Training loss: 0.1405
Epoch: 3/100... Training loss: 0.1376
Epoch: 3/100... Training loss: 0.1398
Epoch: 3/100... Training loss: 0.1411
Epoch: 3/100... Training loss: 0.1437
Epoch: 3/100... Training loss: 0.1410
Epoch: 3/100... Training loss: 0.1437
Epoch: 3/100... Training loss: 0.1427
Epoch: 3/100

Epoch: 4/100... Training loss: 0.1312
Epoch: 4/100... Training loss: 0.1326
Epoch: 4/100... Training loss: 0.1315
Epoch: 4/100... Training loss: 0.1358
Epoch: 4/100... Training loss: 0.1351
Epoch: 4/100... Training loss: 0.1366
Epoch: 4/100... Training loss: 0.1386
Epoch: 4/100... Training loss: 0.1318
Epoch: 4/100... Training loss: 0.1382
Epoch: 4/100... Training loss: 0.1402
Epoch: 4/100... Training loss: 0.1343
Epoch: 4/100... Training loss: 0.1321
Epoch: 4/100... Training loss: 0.1278
Epoch: 4/100... Training loss: 0.1392
Epoch: 4/100... Training loss: 0.1303
Epoch: 4/100... Training loss: 0.1385
Epoch: 4/100... Training loss: 0.1328
Epoch: 4/100... Training loss: 0.1321
Epoch: 4/100... Training loss: 0.1365
Epoch: 4/100... Training loss: 0.1292
Epoch: 4/100... Training loss: 0.1330
Epoch: 4/100... Training loss: 0.1335
Epoch: 4/100... Training loss: 0.1347
Epoch: 4/100... Training loss: 0.1355
Epoch: 4/100... Training loss: 0.1364
Epoch: 4/100... Training loss: 0.1359
Epoch: 4/100

...

Epoch: 8/100... Training loss: 0.1198
Epoch: 8/100... Training loss: 0.1222
Epoch: 8/100... Training loss: 0.1178
Epoch: 8/100... Training loss: 0.1159
Epoch: 8/100... Training loss: 0.1214
Epoch: 8/100... Training loss: 0.1172
Epoch: 8/100... Training loss: 0.1229
Epoch: 8/100... Training loss: 0.1219
Epoch: 8/100... Training loss: 0.1225
Epoch: 8/100... Training loss: 0.1248
Epoch: 8/100... Training loss: 0.1174
Epoch: 8/100... Training loss: 0.1218
Epoch: 8/100... Training loss: 0.1197
Epoch: 8/100... Training loss: 0.1202
Epoch: 8/100... Training loss: 0.1227
Epoch: 8/100... Training loss: 0.1260
Epoch: 8/100... Training loss: 0.1253
Epoch: 8/100... Training loss: 0.1210
Epoch: 8/100... Training loss: 0.1199
Epoch: 8/100... Training loss: 0.1211
Epoch: 8/100... Training loss: 0.1231
Epoch: 8/100... Training loss: 0.1208
Epoch: 8/100... Training loss: 0.1183
Epoch: 8/100... Training loss: 0.1192
Epoch: 8/100... Training loss: 0.1185
Epoch: 8/100... Training loss: 0.1181
Epoch: 8/100

Epoch: 8/100... Training loss: 0.1184
Epoch: 8/100... Training loss: 0.1178
Epoch: 8/100... Training loss: 0.1227
Epoch: 8/100... Training loss: 0.1178
Epoch: 8/100... Training loss: 0.1209
Epoch: 8/100... Training loss: 0.1193
Epoch: 8/100... Training loss: 0.1153
Epoch: 8/100... Training loss: 0.1190
Epoch: 8/100... Training loss: 0.1196
Epoch: 8/100... Training loss: 0.1166
Epoch: 8/100... Training loss: 0.1161
Epoch: 8/100... Training loss: 0.1198
Epoch: 8/100... Training loss: 0.1154
Epoch: 8/100... Training loss: 0.1191
Epoch: 8/100... Training loss: 0.1175
Epoch: 8/100... Training loss: 0.1206
Epoch: 8/100... Training loss: 0.1152
Epoch: 8/100... Training loss: 0.1143
Epoch: 8/100... Training loss: 0.1205
Epoch: 8/100... Training loss: 0.1219
Epoch: 8/100... Training loss: 0.1230
Epoch: 8/100... Training loss: 0.1181
Epoch: 8/100... Training loss: 0.1199
Epoch: 8/100... Training loss: 0.1211
Epoch: 8/100... Training loss: 0.1181
Epoch: 8/100... Training loss: 0.1198
Epoch: 8/100

Epoch: 9/100... Training loss: 0.1184
Epoch: 9/100... Training loss: 0.1170
Epoch: 9/100... Training loss: 0.1185
Epoch: 9/100... Training loss: 0.1183
Epoch: 9/100... Training loss: 0.1155
Epoch: 9/100... Training loss: 0.1183
Epoch: 9/100... Training loss: 0.1147
Epoch: 9/100... Training loss: 0.1204
Epoch: 9/100... Training loss: 0.1157
Epoch: 9/100... Training loss: 0.1209
Epoch: 9/100... Training loss: 0.1175
Epoch: 9/100... Training loss: 0.1220
Epoch: 9/100... Training loss: 0.1169
Epoch: 9/100... Training loss: 0.1171
Epoch: 9/100... Training loss: 0.1189
Epoch: 9/100... Training loss: 0.1185
Epoch: 9/100... Training loss: 0.1225
Epoch: 9/100... Training loss: 0.1219
Epoch: 9/100... Training loss: 0.1201
Epoch: 9/100... Training loss: 0.1195
Epoch: 9/100... Training loss: 0.1188
Epoch: 9/100... Training loss: 0.1174
Epoch: 9/100... Training loss: 0.1175
Epoch: 9/100... Training loss: 0.1196
Epoch: 9/100... Training loss: 0.1158
Epoch: 9/100... Training loss: 0.1187
Epoch: 9/100

Epoch: 10/100... Training loss: 0.1193
Epoch: 10/100... Training loss: 0.1172
Epoch: 10/100... Training loss: 0.1182
Epoch: 10/100... Training loss: 0.1194
Epoch: 10/100... Training loss: 0.1176
Epoch: 10/100... Training loss: 0.1195
Epoch: 10/100... Training loss: 0.1199
Epoch: 10/100... Training loss: 0.1175
Epoch: 10/100... Training loss: 0.1137
Epoch: 10/100... Training loss: 0.1180
Epoch: 10/100... Training loss: 0.1169
Epoch: 10/100... Training loss: 0.1216
Epoch: 10/100... Training loss: 0.1148
Epoch: 10/100... Training loss: 0.1206
Epoch: 10/100... Training loss: 0.1146
Epoch: 10/100... Training loss: 0.1170
Epoch: 10/100... Training loss: 0.1220
Epoch: 10/100... Training loss: 0.1165
Epoch: 10/100... Training loss: 0.1177
Epoch: 10/100... Training loss: 0.1145
Epoch: 10/100... Training loss: 0.1177
Epoch: 10/100... Training loss: 0.1142
Epoch: 10/100... Training loss: 0.1147
Epoch: 10/100... Training loss: 0.1188
Epoch: 10/100... Training loss: 0.1143
Epoch: 10/100... Training

Epoch: 11/100... Training loss: 0.1130
Epoch: 11/100... Training loss: 0.1161
Epoch: 11/100... Training loss: 0.1191
Epoch: 11/100... Training loss: 0.1145
Epoch: 11/100... Training loss: 0.1153
Epoch: 11/100... Training loss: 0.1130
Epoch: 11/100... Training loss: 0.1141
Epoch: 11/100... Training loss: 0.1148
Epoch: 11/100... Training loss: 0.1178
Epoch: 11/100... Training loss: 0.1126
Epoch: 11/100... Training loss: 0.1137
Epoch: 11/100... Training loss: 0.1127
Epoch: 11/100... Training loss: 0.1141
Epoch: 11/100... Training loss: 0.1176
Epoch: 11/100... Training loss: 0.1161
Epoch: 11/100... Training loss: 0.1161
Epoch: 11/100... Training loss: 0.1167
Epoch: 11/100... Training loss: 0.1168
Epoch: 11/100... Training loss: 0.1136
Epoch: 11/100... Training loss: 0.1192
Epoch: 11/100... Training loss: 0.1165
Epoch: 11/100... Training loss: 0.1169
Epoch: 11/100... Training loss: 0.1136
Epoch: 11/100... Training loss: 0.1130
Epoch: 11/100... Training loss: 0.1174
Epoch: 11/100... Training

# 4.栈式自编码器
(1)构建并训练栈式自编码
(2)微调栈式自编码器
(3)将栈式自编码器改进为栈式卷积自编码器

# 5.使用TensorFlow构建与训练SAE模型，完成cifar10图像检索任务

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
cifar10 = input_data.read_data_sets("", one_hot=True)

In [ ]:
cifar10.maybe_download_and_extract()

In [ ]:
#初始化权重函数
def variable_with_weight_loss(shape,std,w1):
    var = tf.Variable(tf.truncated_normal(shape,stddev=std),dtype=tf.float32)
    if w1 is not None:
        weight_loss = tf.multiply(tf.nn.l2_loss(var),w1,name="weight_loss")
        tf.add_to_collection("losses",weight_loss)
    return var

In [ ]:
#获取数据增强后的训练集数据
images_train,labels_train = cifar10_input.distorted_inputs(cifar10_dir,batch_size)
#获取数据裁剪后的测试数据
images_test,labels_test = cifar10_input.inputs(eval_data=True,data_dir=cifar10_dir
                                                   ,batch_size=batch_size)

In [ ]:
#设计第一层卷积
    weight1 = variable_with_weight_loss(shape=[5,5,3,64],std=5e-2,w1=0)
    kernel1 = tf.nn.conv2d(image_holder,weight1,[1,1,1,1],padding="SAME")
    bais1 = tf.Variable(tf.constant(0.0,dtype=tf.float32,shape=[64]))
    conv1 = tf.nn.relu(tf.nn.bias_add(kernel1,bais1))
    pool1 = tf.nn.max_pool(conv1,[1,3,3,1],[1,2,2,1],padding="SAME")，
    norm1 = tf.nn.lrn(pool1,4,bias=1.0,alpha=0.001 / 9,beta=0.75)
#设计第二层卷积
    weight2 = variable_with_weight_loss(shape=[5,5,64,64],std=5e-2,w1=0)
    kernel2 = tf.nn.conv2d(norm1,weight2,[1,1,1,1],padding="SAME")
    bais2 = tf.Variable(tf.constant(0.1,dtype=tf.float32,shape=[64]))
    conv2 = tf.nn.relu(tf.nn.bias_add(kernel2,bais2))
    norm2 = tf.nn.lrn(conv2,4,bias=1.0,alpha=0.01 / 9,beta=0.75)
    pool2 = tf.nn.max_pool(norm2,[1,3,3,1],[1,2,2,1],padding="SAME")
 #第一层全连接层
    reshape = tf.reshape(pool2,[batch_size,-1])
    dim = reshape.get_shape()[1].value
    weight3 = variable_with_weight_loss([dim,384],std=0.04,w1=0.004)
    bais3 = tf.Variable(tf.constant(0.1,shape=[384],dtype=tf.float32))
    local3 = tf.nn.relu(tf.matmul(reshape,weight3)+bais3)
#第二层全连接层
    weight4 = variable_with_weight_loss([384,192],std=0.04,w1=0.004)
    bais4 = tf.Variable(tf.constant(0.1,shape=[192],dtype=tf.float32))
    local4 = tf.nn.relu(tf.matmul(local3,weight4)+bais4)
 #最后一层
    weight5 = variable_with_weight_loss([192,10],std=1/192.0,w1=0)
    bais5 = tf.Variable(tf.constant(0.0,shape=[10],dtype=tf.float32))
    logits = tf.add(tf.matmul(local4,weight5),bais5)

#损失函数
def loss_func(logits,labels):
    labels = tf.cast(labels,tf.int32)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,
                           labels=labels,name="cross_entropy_per_example")
    cross_entropy_mean = tf.reduce_mean(tf.reduce_sum(cross_entropy))
    tf.add_to_collection("losses",cross_entropy_mean)
    return tf.add_n(tf.get_collection("losses"),name="total_loss")
